In [17]:
import redis 

client = redis.Redis('host.docker.internal', 6379, 0)

client.flushdb()

True

In [18]:
client.close()

In [19]:
import json
import numpy as np
import threading
import time
from concurrent.futures import ThreadPoolExecutor
from pcomp.kafka_handlers import KafkaProducerHandler, KafkaConsumerHandler, KafkaConsumerHandlerNeuron
from pcomp.activation_functions import ACTIVATIONS, relu, softmax
from pcomp.redis_utils import RedisHandler
from pcomp.parser import parse_layer_coordinator_message, parse_layer_message
from pcomp.dotmodule import dot_with_bias
from pcomp.neurons_accumulator import NeuronsAccumulator
from pcomp.s3client import S3Client
from io import StringIO

# Kafka Configuration
KAFKA_BROKER = 'kafka:29092'

In [20]:
class Neuron(threading.Thread):
    def __init__(self, layer_id, neuron_id, weights, bias, activation, is_final_layer):
        threading.Thread.__init__(self)
        self.layer_id = layer_id
        self.layer_id_num = int(self.layer_id.replace("layer_", ""))
        self.neuron_id = neuron_id
        self.weights = np.array(weights)
        self.bias = np.array(bias)
        self.is_final_layer = is_final_layer
        self.activation = None if self.is_final_layer else activation
        self.activation_func = None if self.is_final_layer else ACTIVATIONS.get(activation, relu)
        self.redis_handler = RedisHandler('host.docker.internal', 6379, 0, 3)
        self.producer = None
        

    def fetch_input(self, image_id):
        key = f"initial_data_{image_id}" if self.layer_id_num == 0 else f"{self.layer_id_num - 1}_{image_id}"
        return np.frombuffer(self.redis_handler.get(key), dtype=np.float64)

    def process_and_send(self, image_id, input_data):
        #output = dot_with_bias(input_data, self.weights, self.bias, self.activation, False)
        z = np.dot(input_data, self.weights) + self.bias
        output = z if self.is_final_layer else self.activation_func(z)
        self.producer.send_with_key(str(image_id), f"{self.neuron_id}|{format(output, '.17g')}")

    def run(self):
        # Instantiate Kafka consumer and producer inside the thread.
        consumer_handler = KafkaConsumerHandler(f'layer-{self.layer_id_num}', KAFKA_BROKER, group_id=f"{self.neuron_id}_{self.layer_id_num}_group")
        consumer = consumer_handler.get_consumer()
        self.producer = KafkaProducerHandler(KAFKA_BROKER, f'layer-{self.layer_id_num}-streams')
        last_msg_time = time.time()
        while True:
            got_message = False
            for msg in consumer.consume(500, timeout=0.2):
                if msg.error():
                    consumer_handler.error_handling(msg)
                message = msg.value().decode('utf-8')
                got_message = True
                last_msg_time = time.time()
                image_id_str = message
                image_id = int(image_id_str)
                try:
                    input_data = self.fetch_input(image_id)
                    self.process_and_send(image_id, input_data)
                except Exception:
                    pass
            if not got_message and (time.time() - last_msg_time > 10):
                consumer.commit()
                consumer.close()
                self.producer.close()
                self.redis_handler.close()
                break


import threading
import time
from concurrent.futures import ThreadPoolExecutor
from io import BytesIO

class NeuronOutput(threading.Thread):
    def __init__(
        self,
        s3_client=None,
        bucket_name="my-bucket",
        prefix="predictions",
        batch_size=200,
        batch_timeout=60  # seconds
    ):
        threading.Thread.__init__(self)
        self.s3_client = s3_client or S3Client("host.docker.internal:9000", "admin", "admin123")
        self.executor = ThreadPoolExecutor(max_workers=4)
        self.bucket_name = bucket_name
        self.prefix = prefix
        self.batch_size = batch_size
        self.batch_timeout = batch_timeout
        self.file_counter = 0
        self.lock = threading.Lock()
        # Initialize batching state
        self._reset_batch()

    def _reset_batch(self):
        self.csv_buffer = BytesIO()
        self.csv_buffer.write(b"image_id,prediction\n")  # CSV header directly as bytes
        self.record_count = 0
        self.last_batch_time = time.time()

    def run(self):
        consumer_handler = KafkaConsumerHandler('layer-output', KAFKA_BROKER, group_id="neuron_output_coord_group")
        consumer = consumer_handler.get_consumer()
        last_msg_time = time.time()

        while True:
            got_message = False
            for msg in consumer.consume(500, timeout=0.2):
                if msg.error():
                    consumer_handler.error_handling(msg)
                    continue

                image_id, prediction = msg.value().decode('utf-8').split("|")
                got_message = True
                last_msg_time = time.time()

                with self.lock:
                    line = f"{image_id},{prediction}\n".encode("utf-8")
                    self.csv_buffer.write(line)
                    self.record_count += 1
                    should_flush = (
                        self.record_count >= self.batch_size or
                        (time.time() - self.last_batch_time > self.batch_timeout)
                    )

                if should_flush:
                    self.flush_batch_async()

            if not got_message and (time.time() - last_msg_time > 15):
                consumer.commit()
                consumer.close()
                with self.lock:
                    if self.record_count > 0:
                        self.flush_batch_async()  # Flush remaining on shutdown
                break

    def flush_batch_async(self):
        with self.lock:
            if self.record_count == 0:
                return

            # Snapshot the buffer reference (BytesIO object)
            buffer_snapshot = self.csv_buffer
            file_name_snapshot = f"{self.prefix}/batch_{self.file_counter:05d}.csv"
            self.file_counter += 1

            # Reset buffer and counters immediately
            self._reset_batch()

        # Submit background flush
        self.executor.submit(self.flush_batch_to_s3, buffer_snapshot, file_name_snapshot)

    def flush_batch_to_s3(self, buffer, file_name):
        try:
            buffer.seek(0)  # Rewind buffer to start before reading
            content = buffer.read()
            print(f"🚀 Uploading {file_name} ({len(content)} bytes) to S3...")
            self.s3_client.put_object(self.bucket_name, file_name, content)
            print(f"✅ Uploaded {file_name} to bucket {self.bucket_name}")
        except Exception as e:
            print(f"❌ Error uploading {file_name} to S3: {e}")



# class NeuronOutput(threading.Thread):
#     def __init__(self):
#         threading.Thread.__init__(self)
#         # self.redis_handler = RedisHandler('host.docker.internal', 6379, 0, 8)
#         self.last_layer_id_num = 1
#         self.executor = ThreadPoolExecutor(max_workers=4)

#     def run(self):
#         consumer_handler = KafkaConsumerHandler(f'layer-output', KAFKA_BROKER, group_id=f"neuron_output_coord_group")
#         consumer = consumer_handler.get_consumer()
#         last_msg_time = time.time()
#         while True:
#             got_message = False
#             for msg in consumer.consume(500, timeout=0.2):
#                 if msg.error():
#                     consumer_handler.error_handling(msg)
#                 message = msg.value().decode('utf-8')
#                 got_message = True
#                 last_msg_time = time.time()
#                 image_id = int(message)
#                 try:
#                     key = f"{self.last_layer_id_num}_{image_id}"
#                     #outputs = np.frombuffer(self.redis_handler.get(key), dtype=np.float64)
#                     #prediction = int(np.argmax(outputs))
#                     #self.redis_handler.hset('streams:predictions', image_id, prediction)
#                 except Exception:
#                     pass
#             if not got_message and (time.time() - last_msg_time > 15):
#                 consumer.commit()
#                 consumer.close()
#                 # self.redis_handler.close()
#                 break

# Load network and dataset
data = json.load(open("node_based_model.json"))

neurons = []

for layer_name, layer_info in data.items():
    neurons += [Neuron(layer_id=layer_name, neuron_id=i, weights=node['weights'], bias=node['biases'], activation=node['activation'], is_final_layer=(layer_name == list(data.keys())[-1])) for i, node in enumerate(layer_info['nodes'])]

neuron_output = NeuronOutput()

# Start all threads
for thread in neurons:
    thread.start()

neuron_output.start()

print("Threads started")

Threads started
🚀 Uploading predictions/batch_00000.csv (1110 bytes) to S3...
✅ Uploaded predictions/batch_00000.csv to bucket my-bucket
🚀 Uploading predictions/batch_00001.csv (1220 bytes) to S3...
✅ Uploaded predictions/batch_00001.csv to bucket my-bucket
🚀 Uploading predictions/batch_00002.csv (1220 bytes) to S3...
✅ Uploaded predictions/batch_00002.csv to bucket my-bucket
🚀 Uploading predictions/batch_00003.csv (1220 bytes) to S3...
✅ Uploaded predictions/batch_00003.csv to bucket my-bucket
🚀 Uploading predictions/batch_00004.csv (1220 bytes) to S3...
✅ Uploaded predictions/batch_00004.csv to bucket my-bucket


In [23]:
import redis

# Connect to Redis
r = RedisHandler('host.docker.internal', 6379, 0)

# Get hashes from Redis
images_label = r.hgetall('images_label')
predictions = r.hgetall('streams:predictions')

# Decode bytes to string
images_label = {k.decode(): v.decode() for k, v in images_label.items()}
predictions = {k.decode(): v.decode() for k, v in predictions.items()}

# Calculate accuracy
correct = 0
total = len(images_label)

for field, label_val in images_label.items():
    pred_val = predictions.get(field, None)
    if pred_val == label_val:
        correct += 1

accuracy = (correct / total) * 100 if total > 0 else 0

print(f'Accuracy: {accuracy:.2f}% ({correct}/{total})')


Accuracy: 95.30% (1906/2000)


In [16]:
# Wait for all threads to complete
for thread in neurons:
    thread.join()

neuron_output.join()

print("Threads finished")

KeyboardInterrupt: 

%4|1745628997.770|MAXPOLL|rdkafka#consumer-831| [thrd:main]: Application maximum poll interval (300000ms) exceeded by 64ms (adjust max.poll.interval.ms for long-running message processing): leaving group
